In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import json 
import cv2
from easydict import EasyDict

from glob import glob
import os

import timm
import torch
from torch import nn as nn
from torchvision import models
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, StratifiedKFold
import albumentations as A


# Label Preprocessing

In [2]:
def label_preprocessing(path) :
    public_dataset_folder = os.listdir(path)

    label_encoder = {}
    for idx, data_name in enumerate(public_dataset_folder) :
        label_encoder[idx] = data_name

    label_decoder = {val:key for key, val in label_encoder.items()}
    return label_encoder, label_decoder

# Transform

In [3]:
def transform(size=224):
    train_transforms = A.Compose([
                A.Resize(size ,size),
                A.OneOf([
                    A.Rotate(),
                    A.HorizontalFlip(),
                    A.VerticalFlip()
                ], p=1)
            ])

    val_transforms = A.Compose([
        A.Resize(size,size)
    ])
    
    return train_transforms, val_transforms

# Data split

In [4]:
def data_split(path, label_decoder, test_size=0.2) : 
    imgs = glob(os.path.join(path, '*/*.jpg'))
    label_list = [label_decoder[img_path.split('\\')[-2]] for img_path in imgs]
    
    return train_test_split(imgs, test_size=test_size, shuffle=True, stratify=label_list)

# Custom Dataset 정의

In [5]:
class VillageDataset(Dataset) :
    def __init__(self, files, label_decoder, transform) :
        super(VillageDataset, self).__init__()
        self.files = files
        self.label_decoder = label_decoder
        self.transform = transform
        
    def __len__(self) :
        return len(self.files)
    
    def __getitem__(self, idx) :
        file_path = self.files[idx]
        
        label = self.label_decoder[file_path.split('\\')[-2]]
        
        img = cv2.imread(file_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        img = self.transform(image=img)['image']
        img = img.transpose(2, 0, 1)
        
        return torch.tensor(img, dtype=torch.float32), torch.tensor(label, dtype=torch.long)

# Cutmix

In [6]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = int(W * cut_rat)
    cut_h = int(H * cut_rat)
 
    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

# Model

In [7]:
class CNN(nn.Module) :
    def __init__(self, model_name, n_classes=38) :
        super(CNN, self).__init__()
        self.model = timm.create_model(model_name, num_classes=n_classes, pretrained=True)
    
    def forward(self, x) :
        x = self.model(x)
        return x
    
def training_setting(opt) : 
    model = CNN(opt.model_name, opt.n_classes)
    model = model.to(opt.device)
    optimizer = torch.optim.Adam(model.parameters(), lr=opt.learning_rate)
    criterion = nn.CrossEntropyLoss()
    
    return model, optimizer, criterion


# Training

In [26]:
def accuracy_function(real, pred):    
    real = real.cpu()
    pred = torch.argmax(pred, dim=1).cpu()
    score = f1_score(real, pred, average='macro')
    return score

def run(train_loader, valid_loader, opt) :
    print("optin : ",opt)
    model, optimizer, criterion = training_setting(opt)
    best_loss = 10
    for epoch in range(opt.epochs) : 

        # training
        tqdm_train = tqdm(train_loader)
        train_loss, train_macro_f1 = 0, 0
        for batch, batch_item in enumerate(tqdm_train) :
            model.train()
            img = batch_item[0].to(opt.device)
            label = batch_item[1].to(opt.device)
            
            lam = np.random.beta(1.0, 1.0)
                        
            optimizer.zero_grad()
            with torch.cuda.amp.autocast():
                # add - cutmix
                rand_index = torch.randperm(img.size()[0])
                target_a = label
                target_b = label[rand_index]
                bbx1, bby1, bbx2, bby2 = rand_bbox(img.size(), lam)
                img[:, :, bbx1:bbx2, bby1:bby2] = img[rand_index, :, bbx1:bbx2, bby1:bby2]
                lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (img.size()[-1] * img.size()[-2]))

                output = model(img)
                loss = criterion(output, target_a) * lam + criterion(output, target_b) * (1. - lam)

            loss.backward()
            optimizer.step()
            score = accuracy_function(label, output)
            
            train_loss = (train_loss + loss.item()) / (batch + 1)
            train_macro_f1 = (train_macro_f1 + score) / (batch + 1)
            
            tqdm_train.set_postfix({"Epoch" : epoch+1,
                                    "train_loss" : train_loss,
                                    "train_f1" : train_macro_f1})
            
#             print(f"Traing Epoch : [{epoch}/{opt.epochs}] loss : {train_loss}  f1 : {train_macro_f1}",end='\r')
            
#         print(f"Traing Epoch : [{epoch}/{opt.epochs}] loss : {train_loss}  f1 : {train_macro_f1}")
        
        # validation
        tqdm_valid = tqdm(valid_loader)
        mean_valid_loss, valid_loss, valid_macro_f1 = 0, 0, 0
        for batch, batch_item in enumerate(tqdm_valid) :
            img = batch_item[0].to(opt.device)
            label = batch_item[1].to(opt.device)
            
            model.eval()
            with torch.no_grad():
                output = model(img)
                loss = criterion(output, label)
            score = accuracy_function(label, output)
            
            valid_loss += loss
            valid_macro_f1 += score
            
            mean_valid_loss = valid_loss / (batch + 1)
            tqdm_valid.set_postfix({"valid_loss" : mean_valid_loss,
                                    "valid_f1" : valid_macro_f1/ (batch + 1)})
            
#             print(f"Valid Epoch : [{epoch}/{opt.epochs}] loss : {valid_loss}  f1 : {valid_macro_f1}",end='\r')
            
#         print(f"Valid Epoch : [{epoch}/{opt.epochs}] loss : {valid_loss}  f1 : {valid_macro_f1}")

        if valid_loss < best_loss :
            best_f1 = valid_macro_f1
            os.makedirs(opt.save_path, exist_ok=True)
            torch.save(model.state_dict(), os.path.join(opt.save_path, f'{epoch}E_{round(valid_loss, 4)}_{opt.model_name}.pt'))

In [29]:
opt = {"public_dataset_path" : "../data/public/PlantVillage",
       "save_path" : "../pretrain",
        "batch_size" : 16,
        "model_name" : 'deit_small_patch16_224',
        "n_classes" : 38,
        "learning_rate" : 1e-4,
        "device" : "cuda",
        "epochs" : 15}

opt = EasyDict(opt)

label_encoder, label_decoder = label_preprocessing(opt.public_dataset_path)
train_transforms, valid_transforms = transform()

train, valid = data_split(opt.public_dataset_path, label_decoder)
# display(len(train))

train_dataset = VillageDataset(train, label_decoder, train_transforms)
valid_dataset = VillageDataset(valid, label_decoder, valid_transforms)

train_loader = DataLoader(train_dataset, batch_size=opt.batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=opt.batch_size, shuffle=False)

In [30]:
run(train_loader, valid_loader, opt)

optin :  {'public_dataset_path': '../data/public/PlantVillage', 'save_path': '../pretrain', 'batch_size': 16, 'model_name': 'deit_small_patch16_224', 'n_classes': 38, 'learning_rate': 0.0001, 'device': 'cuda', 'epochs': 15}


 51%|██████████████▉              | 1394/2716 [02:54<02:45,  8.00it/s, Epoch=2, train_loss=0.000864, train_f1=0.000386]


KeyboardInterrupt: 